# 6강) FAISS 실습

### Requirements

In [ ]:
!pip install datasets==2.7.0 -q
!pip install transformers==4.24.0 -q
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━

## 5강의 자료를 활용해 passage / question enocoder 학습


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)

torch.manual_seed(2023)
torch.cuda.manual_seed(2023)
np.random.seed(2023)
random.seed(2023)

In [ ]:
dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Training Dataset 준비

In [ ]:
# Use subset (10 example) of original training dataset 
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]

# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break

In [ ]:
q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')

max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

BERT encoder 학습시키기

In [ ]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      pooled_output = outputs[1]
      return pooled_output


def train(args, num_neg, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)  #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()
  return p_model, q_model

In [ ]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)

In [ ]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.transf

Train function 정의 후, 두개의 encoder fine-tuning 하기


In [ ]:
p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


Iteration:   0%|          | 0/10 [00:00<?, ?it/s]


Iteration:  10%|█         | 1/10 [00:04<00:36,  4.05s/it]


Iteration:  20%|██        | 2/10 [00:05<00:18,  2.29s/it]


Iteration:  30%|███       | 3/10 [00:06<00:12,  1.74s/it]


Iteration:  40%|████      | 4/10 [00:07<00:08,  1.47s/it]


Iteration:  50%|█████     | 5/10 [00:08<00:06,  1.32s/it]


Iteration:  60%|██████    | 6/10 [00:09<00:04,  1.23s/it]


Iteration:  70%|███████   | 7/10 [00:10<00:03,  1.17s/it]


Iteration:  80%|████████  | 8/10 [00:11<00:02,  1.14s/it]


Iteration:  90%|█████████ | 9/10 [00:12<00:01,  1.11s/it]


Iteration: 100%|██████████| 10/10 [00:13<00:00,  1

## Passage retrieval 준비하기

Search corpus: KorQuAD validation context

In [ ]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

Passage encoder를 활용하여 passage dense embedding 생성

In [ ]:
eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)

Iteration: 100%|██████████| 120/120 [00:27<00:00,  4.36it/s]


(960, 768)

Question encoder를 활용해여 quesntion dense embedding 생성

In [ ]:
sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['1960년대에 스포츠카나 프로토 중심으로 열린 경기는?',
 '정도전이 실시한 개혁과 요동정벌은 누구의 반발을 불러일으키는가?',
 '마리 앙투아네트에게 생긴 비난 어린 별명은 무엇인가?',
 '천안함 침몰 사건 이후 몇 개월 만에 연평도 포격사건이 벌어졌는가?',
 '아이케 장군이 다시 지휘권을 받은 달은 언제인가?']

In [ ]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

## GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [ ]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [ ]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[245, 579, 637,  ..., 489, 686, 504],
        [245, 637, 442,  ..., 504, 686, 489],
        [245, 579, 637,  ..., 686, 504, 489],
        [245, 637, 579,  ..., 686, 489, 504],
        [245, 579, 893,  ..., 489, 686, 504]], device='cuda:0')
--- 0.011309623718261719 seconds ---


In [ ]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 1960년대에 스포츠카나 프로토 중심으로 열린 경기는? 

[Ground truth passage]
뉘르부르크링의 역사는 순조롭게 흘러가는 듯 했으나 제2차 세계 대전이 일어나면서, 1940년대에는 제대로 된 경기가 이루어지지 못했다. 하지만 2차대전 종전 후 뉘르부르크링은 황금기에 접어든다. 종전 후 트랙의 보수 공사가 이뤄졌고 1951년 FIA의 공식 인증을 받아 포뮬러 원 트랙으로 선정된다. 동시에 유럽 그랑프리, 뉘르부르크링 1000 km 등 각종 대규모 경기도 개최되었다. 게다가 후안 마누엘 판지오(Juan Manuel Fangio), 재키 스튜어트(Jackie Stewart)와 같은 전설적인 드라이버의 등장으로, 뉘르부르크링의 국제적 관심과 인기는 날로 높아만 갔다. 1960년대에는 스포츠카와 프로토 타입을 위한 투어링 카 챔피언십이 신설됐다. 1961년에는 10만명의 관중이 운집하는 대성황을 이루었다. 

Top-1 passage with score 40.7131
《포켓몬스터 크리스탈》은 비평가들에게 꽤 좋은 평가를 받았으며, 게임랭킹스에서는 총 80%의 집계 점수를 얻었다. 그러나 대부분의 평가에서는 《포켓몬스터 금·은》 이외에 추가된 새로운 요소와 특징들이 의미있을 정도로 충분하지 않다는 의견이 많았다. IGN의 크레이그 해리스는 이 게임에 "눈에 띄는" 등급인 10점 만점에 9점을 주면서 다음과 같이 말하였다. "이 마지막(이길 바라는) 게임보이 컬러판은 정확히 말해 자신이 이전 게임을 구매한 10억 명의 게이머 중 한 명이 아닐 때 구입하는 것이며, 디자인과 그래픽에 약간의 업데이트가 더해진 버전이다. 그러나 이러한 특징은 이미 이전 게임을 한 개, 혹은 두 개 소유한 사람들에게 "반드시 사야하는" 것으로 만들기에는 충분치 않다." 일본 잡지 《패미통》에서는 게임에 40점 만점에 34점의 점수를 매겼다.
Top-2 passage with score 37.6332
자신의 미니홈피 계정에 직접 쓴 다수의 시와 수필을

## FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [ ]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

In [ ]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [ ]:
# 3. Search using indexer
start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0010409355163574219 seconds ---


In [ ]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[24.777351 27.392805 29.592102 29.595135 30.064186]
 [30.360569 32.88949  33.415215 33.555977 34.0213  ]
 [24.02727  26.624054 28.137016 28.610079 29.055676]
 [31.789742 34.28707  35.328056 35.87351  36.13344 ]
 [30.661829 33.407127 35.58259  35.76271  36.053757]]


=======[Index of Top-5 Passages]=======
[[727 640 784 492  76]
 [727 640 784 492 864]
 [727 640 492 784  76]
 [727 640 784 492 864]
 [727 640 492 784 864]]


In [ ]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
  print('\n')

[Search query]
 1960년대에 스포츠카나 프로토 중심으로 열린 경기는? 

[Ground truth passage]
뉘르부르크링의 역사는 순조롭게 흘러가는 듯 했으나 제2차 세계 대전이 일어나면서, 1940년대에는 제대로 된 경기가 이루어지지 못했다. 하지만 2차대전 종전 후 뉘르부르크링은 황금기에 접어든다. 종전 후 트랙의 보수 공사가 이뤄졌고 1951년 FIA의 공식 인증을 받아 포뮬러 원 트랙으로 선정된다. 동시에 유럽 그랑프리, 뉘르부르크링 1000 km 등 각종 대규모 경기도 개최되었다. 게다가 후안 마누엘 판지오(Juan Manuel Fangio), 재키 스튜어트(Jackie Stewart)와 같은 전설적인 드라이버의 등장으로, 뉘르부르크링의 국제적 관심과 인기는 날로 높아만 갔다. 1960년대에는 스포츠카와 프로토 타입을 위한 투어링 카 챔피언십이 신설됐다. 1961년에는 10만명의 관중이 운집하는 대성황을 이루었다. 

Top-1 passage with distance 24.7774
전반적인 디자인은 메탈과 유리 소재를 사용하여 갤럭시 S6 & 갤럭시 S6 엣지와 패밀리룩을 이루고 있다. 기본 색상은 블랙 사파이어, 화이트 펄, 골드 플래티넘, 실버 티타늄으로 총 4종이다. AP 사양은 갤럭시 S6 & 갤럭시 S6 엣지와 동일한 삼성 엑시노스 7420을 사용한다. ARM Cortex-A57 쿼드코어 CPU와 ARM Cortex-A53 쿼드코어 CPU에 big.LITTLE 솔루션을 적용한 HMP 모드 지원 옥타코어 CPU와 ARM Mali-T760 옥타코어 GPU를 사용한다. RAM은 LPDDR4 SDRAM 방식이며 4 GB다. 이는 삼성전자의 모바일 기기 중 갤럭시 S6 엣지+와 같이 최초로 4 GB RAM을 탑재했다. 내장 메모리는 32 GB와 64 GB, 128GB(실버 티타늄 및 골드 플래티넘만 128GB)& 갤럭시 S6 엣지와 동일하게 UFS 2.0 규격을 사용한다. 그러나 확장성의 경우 micro SD 카드 슬롯이 존재하지 

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

